# **Load The Data** 

In [ ]:
import pandas as pd
import numpy as np
# Load pickled data
import pickle
from keras.datasets import cifar10
import warnings
warnings.filterwarnings('ignore')

In [ ]:

(X_train, y_train), (X_test, y_test) = cifar10.load_data()
# y_train.shape is 2d, (50000, 1). While Keras is smart enough to handle this
# it's a good idea to flatten the array.
y_train = y_train.reshape(-1)
y_test = y_test.reshape(-1)

# **Splitting Data**

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=42, stratify = y_train)
n_classes = len(pd.Series(y_train).unique())

In [ ]:
!pip install --upgrade tf_slim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 352 kB 3.2 MB/s 


# **Model Architecture**

In [ ]:
import tensorflow as tf
import tf_slim as slim
tf.compat.v1.disable_eager_execution()

In [ ]:
def LeNet(x): 
    # Arguments used for tf.truncated_normal, randomly defines variables for the weights and biases for each layer
    mu = 0
    sigma = 0.1
    
    # Layer 1: Convolutional. Input = 32x32x3. Output = 28x28x6.
    conv1_W = tf.Variable(tf.compat.v1.truncated_normal(shape=(5, 5, 3, 6), mean = mu, stddev = sigma))
    conv1_b = tf.Variable(tf.zeros(6))
    conv1   = tf.nn.conv2d(x, conv1_W, strides=[1, 1, 1, 1], padding='VALID') + conv1_b
    
    # Activation 1. using relu Activation Function
    conv1 = tf.nn.relu(conv1)

    # Max Pooling. Input = 28x28x6. Output = 14x14x6.
    conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
    
    
    # Layer 2: Convolutional. Input = 14x14x6. Output = 10x10x16.
    conv2_W = tf.Variable(tf.compat.v1.truncated_normal(shape=(5, 5, 6, 16), mean = mu, stddev = sigma))
    conv2_b = tf.Variable(tf.zeros(16))
    conv2   = tf.nn.conv2d(conv1, conv2_W, strides=[1, 1, 1, 1], padding='VALID') + conv2_b
    
    # Activation 2.
    conv2 = tf.nn.relu(conv2)

    # Pooling. Input = 10x10x16. Output = 5x5x16.
    conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
    
    #Flatten Input : reduces the input data into a single dimension 5x5x16 = 400
    # Flatten. Input = 5x5x16. Output = 400.
    flattened   = slim.flatten(conv2)
    
    #Matrix multiplication
    #input: 1x400
    #weight: 400x120 
    #Matrix multiplication(dot product rule)
    #output = 1x400 * 400*120 => 1x120
    
     # Layer 3: Fully Connected. Input = 400. Output = 120.
    fullyc1_W = tf.Variable(tf.compat.v1.truncated_normal(shape=(400, 120), mean = mu, stddev = sigma))
    fullyc1_b = tf.Variable(tf.zeros(120))
    fullyc1   = tf.matmul(flattened, fullyc1_W) + fullyc1_b
    
    # Full connected layer activation 1.
    fullyc1    = tf.nn.relu(fullyc1)
    
    # Layer 4: Fully Connected. Input = 120. Output = 84.
    fullyc2_W  = tf.Variable(tf.compat.v1.truncated_normal(shape=(120, 84), mean = 0, stddev = 0.1))
    fullyc2_b  = tf.Variable(tf.zeros(84))
    fullyc2    = tf.matmul(fullyc1, fullyc2_W) + fullyc2_b
    
    # Full connected layer activation 2.
    fullyc2    = tf.nn.relu(fullyc2)
    
    # Layer 5: Fully Connected. Input = 84. Output = 43.
    fullyc3_W  = tf.Variable(tf.compat.v1.truncated_normal(shape=(84, 43), mean = 0, stddev = 0.1))
    fullyc3_b  = tf.Variable(tf.zeros(43))
    logits = tf.matmul(fullyc2, fullyc3_W) + fullyc3_b
    
    return logits

print("Model defined.")

Model defined.


# **Features and Labels**
x is a placeholder for a batch of input images. y is a placeholder for a batch of output labels. .

In [ ]:
x = tf.compat.v1.placeholder(tf.float32, (None, 32, 32, 3))
y = tf.compat.v1.placeholder(tf.int64, (None))
one_hot_y = tf.one_hot(y, n_classes)

In [ ]:
"""      
Hyper parameters

"""

#learning rates745
learning_rate = 0.001
#the number of times the algorithm sees the entire data set
epochs = 42
#the number of samples (inputs) that will be passed through to the network at one time.
batch_size = 128


# **Evaluate the Model**

In [ ]:
logits = LeNet(x)
#relation between probabilities and Erreur function CE
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits( logits=logits, labels=y)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate = learning_rate)
training_operation = optimizer.minimize(loss_operation)
predict_operation = tf.argmax(logits, 1)
predict_proba_operation = tf.nn.softmax(logits=logits)

In [ ]:
correct_prediction = tf.equal(predict_operation, tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    total_loss = 0
    sess = tf.compat.v1.get_default_session()
    for offset in range(0, num_examples, batch_size):
        batch_x, batch_y = X_data[offset:offset+batch_size], y_data[offset:offset+batch_size]
        loss, accuracy = sess.run([loss_operation, accuracy_operation], feed_dict={x: batch_x, y: batch_y})
        total_accuracy += (accuracy * len(batch_x))
        total_loss += (loss * len(batch_x))
    return total_loss/num_examples, total_accuracy/num_examples

print("Evaluate function defined.")

Evaluate function defined.


In [ ]:
from sklearn.utils import shuffle
from time import time
import logging, datetime

In [ ]:

"""
Log File for Metadata 
"""
logger = logging.getLogger()
#create file handler for logger
def setup_file_logger(log_file):
    hdlr = logging.FileHandler(log_file)
    formatter = logging.Formatter('%(asctime)s %(levelname)s %(message)s')
    hdlr.setFormatter(formatter)
    logger.addHandler(hdlr) 
    logger.setLevel(logging.INFO)

def log(message):
    #outputs to Colab console
    print('{} {}'.format(datetime.datetime.now(), message))
    #outputs to file
    logger.info(message)

setup_file_logger('training_Modul.log')

# **Train the Model**

In [ ]:
validation_accuracy = []
validation_loss = []
training_accuracy = []
training_loss = []
saver1 = tf.compat.v1.train.Saver()
with tf.compat.v1.Session() as sess:
    sess.run(tf.compat.v1.global_variables_initializer())
    num_examples = len(X_train)
    
    print("Start Training...")
    print("Number of epochs : {}".format(epochs))
    print("Batch size : {}".format(batch_size))
    print("Learning rate : {}".format(learning_rate))
    print()
    

    for i in range(epochs):
      try:
            X_train, y_train = shuffle(X_train, y_train)
            for offset in range(0, num_examples, batch_size):
                end = offset + batch_size
                batch_x, batch_y = X_train[offset:end], y_train[offset:end]
                sess.run(training_operation, feed_dict={x: batch_x, y: batch_y})
                        
            log("EPOCH {} ...".format(i+1))
            # Training data
            train_loss, train_accuracy = evaluate(X_train, y_train)
            training_accuracy.append(train_accuracy)
            training_loss.append(train_loss)
            log("Training Accuracy = {:.3f}".format(train_accuracy))
            print("Training error = {:.3f} ".format( train_loss ))
            # Validation data
            valid_loss, valid_accuracy = evaluate(X_valid, y_valid)
            validation_accuracy.append(valid_accuracy)
            validation_loss.append(valid_loss)
            log("Validation Accuracy = {:.3f}".format(valid_accuracy))
            print("Validation error = {:.3f}".format(valid_loss))
            print()


      except KeyboardInterrupt:
          test_loss, test_accuracy = evaluate(X_valid, y_valid)
          log('Accuracy Model On Test Images: {}'.format(test_accuracy))
          print('Loss Model On Test Images: {}'.format(test_loss))   
          break

        
    saver1.save(sess, 'LeNet_Modul')
    print("Model saved")

Start Training...
Number of epochs : 42
Batch size : 128
Learning rate : 0.001



INFO:root:EPOCH 1 ...


2022-09-03 16:07:06.863009 EPOCH 1 ...


INFO:root:Training Accuracy = 0.290


2022-09-03 16:07:18.799158 Training Accuracy = 0.290
Training error = 1.950 


...

2022-09-03 16:13:05.108686 Training Accuracy = 0.562
Training error = 1.230 


INFO:root:Validation Accuracy = 0.488


2022-09-03 16:13:10.764279 Validation Accuracy = 0.488
Validation error = 1.468



INFO:root:Accuracy Model On Test Images: 0.49466666668256126


2022-09-03 16:13:28.543065 Accuracy Model On Test Images: 0.49466666668256126
Loss Model On Test Images: 1.438669055366516
Model saved
